# Notebook base de funciones utilizadas generalmente en todos los modelos:
### Este Notebook está diseñado con el propósito de generar funciones que se podrán utilizar en el desarrollo del ejercicio que modifiquen el dataset.

In [1]:
#Importamos los paquetes a utilizar

In [2]:
import boto3
import io
from PIL import Image, ImageDraw

import pandas as pd
import random
             
from IPython.display import display 
import json
import plotly.express as px
import os
from botocore import UNSIGNED
from botocore.client import Config

import seaborn as sn
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


# Constants:

Aquí definimos variables constantes para la extracción y modificación de los datos:

In [3]:
# Bucket en S3 habilitado para el almacenamiento de las imágenes y las características de las mismas:
BUCKET = 'test-bucket'
# Region en AWS:
REGION_NAME = 'test-region-name'
#Versión de la partición a utilizar:
LATEST_CSV = 'dataset.csv'

# Utilitary Functions:

Definimos funciones que nos serán de utilizad en el resto de los notebooks:

In [2]:
# Pre procesamiento para limpiar la base de datos del dataset actualizado:


def prepro():
    df_facturas = pd.read_csv(LATEST_CSV, index_col=0)
    #PRIMERO VAMOS A PONERLAS EN MAYÚSCULAS
    df_facturas['issuer'] = df_facturas['issuer'].str.strip(' ')
    df_facturas['issuer'] = df_facturas['issuer'].str.upper()

    #REEMPLAZAMOS LAS TILDES POR EL CARACTER
    df_facturas['issuer'] = df_facturas['issuer'].replace({'Á':'A'}, regex=True)
    df_facturas['issuer'] = df_facturas['issuer'].replace({'É':'E'}, regex=True)
    df_facturas['issuer'] = df_facturas['issuer'].replace({'Í':'I'}, regex=True)
    df_facturas['issuer'] = df_facturas['issuer'].replace({'Ó':'O'}, regex=True)
    df_facturas['issuer'] = df_facturas['issuer'].replace({'Ú':'U'}, regex=True)

    #REEMPLAZAMOS EL DOBLE ESPACIO
    df_facturas['issuer'] = df_facturas['issuer'].replace({'  ':' '}, regex=True)

    #ELIMINAMOS ESAS PALABRAS
    df_facturas['issuer'] = df_facturas['issuer'].replace({' LTDA.':''}, regex=True)
    df_facturas['issuer'] = df_facturas['issuer'].replace({' LTDA':''}, regex=True)
    df_facturas['issuer'] = df_facturas['issuer'].replace({' SAS':''}, regex=True)
    df_facturas['issuer'] = df_facturas['issuer'].replace({' S.A.S':''}, regex=True)
    df_facturas['issuer'] = df_facturas['issuer'].replace({' S.A.S.':''}, regex=True)
    df_facturas['issuer'] = df_facturas['issuer'].replace({' S. A. S.':''}, regex=True)
    df_facturas['issuer'] = df_facturas['issuer'].replace({' S. A. S':''}, regex=True)
    df_facturas['issuer'] = df_facturas['issuer'].replace({' T.A.':''}, regex=True)
    df_facturas['issuer'] = df_facturas['issuer'].replace({' SA':''}, regex=True)
    df_facturas['issuer'] = df_facturas['issuer'].replace({' S.A.':''}, regex=True)
    df_facturas['issuer'] = df_facturas['issuer'].replace({' S.A':''}, regex=True)

    #ARREGLAMOS EL DATASET CON PALABRAS PARA DEJAR UNA BUENA CALIDAD DE IFO
    df_facturas['issuer'] = df_facturas['issuer'].replace({'BBI COLOMBIA.':'BBI COLOMBIA'}, regex=True)
    df_facturas['issuer'] = df_facturas['issuer'].replace({'BBI COLOMBIA.':'BBI COLOMBIA'}, regex=True)
    df_facturas['issuer'] = df_facturas['issuer'].replace({'CADENA COMERCIAL OXXO COLOMBIA':'CADENA COMERCIAL OXXO'}, regex=True)
    df_facturas['issuer'] = df_facturas['issuer'].replace({'CADENA COMERCIAL OXXO':'CADENA COMERCIAL OXXO COLOMBIA'}, regex=True)
    df_facturas['issuer'] = df_facturas['issuer'].replace({'CARILLA ':'CARULLA '}, regex=True)
    df_facturas['issuer'] = df_facturas['issuer'].replace({'COLDANITAS':'COLSANITAS'}, regex=True)
    df_facturas['issuer'] = df_facturas['issuer'].replace({'COLDANITAS':'COLSANITAS'}, regex=True)
    df_facturas['issuer'] = df_facturas['issuer'].replace({'D1.':'D1'}, regex=True)
    df_facturas['issuer'] = df_facturas['issuer'].replace({'D1':'D1.'}, regex=True)
    df_facturas['issuer'] = df_facturas['issuer'].replace({'EPSNITAS.':'EPSNITAS'}, regex=True)
    df_facturas['issuer'] = df_facturas['issuer'].replace({'EPSNITAS':'EPSNITAS.'}, regex=True)
    
    df_facturas['issuer'] = df_facturas['issuer'].replace({ 'DORANDO COLOMBIA': 'DOTBAND COLOMBIA'}, regex=True)
    
    df_facturas['issuer'] = df_facturas['issuer'].replace({  'WINGION': 'WING STATION'}, regex=True)

    df_facturas['issuer'] = df_facturas['issuer'].replace({"MCDONALD'S":'ARCOS DORADOS COLOMBIA'}, regex=True)
    df_facturas['issuer'] = df_facturas['issuer'].replace({'MCDONALDS': 'ARCOS DORADOS COLOMBIA'}, regex=True)

    df_facturas['issuer'] = df_facturas['issuer'].replace({'FRUTIVER': 'FRUTIVERDURAS CAMACHO'}, regex=True)
    df_facturas['issuer'] = df_facturas['issuer'].replace({'FRUTIVERDURAS CAMACH9': 'FRUTIVERDURAS CAMACHO'}, regex=True)

    df_facturas['issuer'] = df_facturas['issuer'].replace({'LOS HORNITOS PASTELERIA Y PANADERIA': 'LOS HORNITOS PANADERIA Y PASTELERIA'}, regex=True)
    df_facturas['issuer'] = df_facturas['issuer'].replace({'MERCADERIA JUSTO Y BUENO':  'MERCADERIA JUSTO & BUENO'}, regex=True)

    df_facturas['issuer'] = df_facturas['issuer'].replace({'SISTEMA POS COLSUBSIDIO':'SISTEMA P. O. S. COLSUBSIDIO'}, regex=True)
    df_facturas['issuer'] = df_facturas['issuer'].replace({'SISTEMA POS COLSUBSUDIO':'SISTEMA P. O. S. COLSUBSIDIO'}, regex=True)

    df_facturas['issuer'] = df_facturas['issuer'].replace({'CIA. MEDIC PREPAG COLSANITAS':'CIA. MEDIC. PREPAG. COLSANITAS'}, regex=True)

    df_facturas['issuer'] = df_facturas['issuer'].replace({'OLIMMPICA': 'OLIMPICA'}, regex=True)
    df_facturas['issuer'] = df_facturas['issuer'].replace({'HAMBURGUESAS EL CORRAK':'HAMBURGUESAS EL CORRAL'}, regex=True)
    df_facturas['issuer'] = df_facturas['issuer'].replace({'FRUTOVERDURAS CAMACHO': 'FRUTY VERDURAS CAMACHO'}, regex=True)

    df_facturas['issuer'] = df_facturas['issuer'].replace({'DISTRIBUIDORA DONA ELENA':'DISTRIBUIDORA DOÑA ELENA'}, regex=True)
    
    df_facturas['total_value'] = df_facturas['total_value'].replace({'$26,823,142':'$26,823.42'}, regex=False)
    df_facturas['total_value'] = df_facturas['total_value'].replace({'$7,877,595':'$78775.95'}, regex=False)
    df_facturas['total_value'] = df_facturas['total_value'].replace({'$7,403,720':'$74037.20'}, regex=False)

    return df_facturas

In [5]:
#Agregamos al columna de categoría:
#Para esto crearemos un diccionaría con las categorías:
diccionario = {
'-':'Alimentación',
 '287-SEVEN/SEVEN CEDRITOS':'Moda',
 '8XITON MARTIN':'Grande superficie',
 'ADIDAS COLOMBIA':'Moda',
 'AFC ADMINISTRADORES DE FRANQUICIAS EN COLOMBIA':'Moda',
 'AG FRUTAL':'Alimentación',
 'AL AGUA PATOS':'Alimentación',
 'ALKOSTO':'Grande superficie',
 'ALLBEROCA':'Moda',
 'AMARILLA CAFE CERAMICO':'Alimentación',
 'AMERICAN EAGLE':'Moda',
 'AMINTA NARA GREEN HILLS':'Alimentación',
 "ANIMAL'S":'Mascotas',
 'ARCHIES':'Alimentación',
 'ARCOS DORADOS COLOMBIA':'Alimentación',
 'ARISTOBULO ROJAS ANGEL':'Alimentación',
 'ARTURO CALLE':'Moda',
 'ASADERO RESTAURANTE LIDER POLLO':'Alimentación',
 'AUDIFARMA AMB':'Salud y Bienestar',
 'AVALCREDITOS':'Compromisos Bancarios',
 'BANCO CAJA SOCIAL':'Compromisos Bancarios',
 'BBC':'Alimentación',
 'BBI COLOMBIA':'Alimentación',
 'BEL-STAR':'Salud y Bienestar',
 'BENDITA CARNE':'Alimentación',
 'BOREO LOUNGE FOOD & DRINKS CARLOS ANDRES JARAMILLO':'Alimentación',
 'BOSI':'Moda',
 'BROT':'Alimentación',
 'BUÑUELERIA ESPECIAL':'Alimentación',
 'CADENA COMERCIAL OXXO COLOMBIA':'Grande superficie',
 'CAFAM DROGUERIAS':'Salud y Bienestar',
 'CAFE CLICHE':'Alimentación',
 'CAFE QUINDIO':'Alimentación',
 'CARMEN':'Alimentación',
 'CARNESN IGNACIO':'Alimentación',
 'CARULLA':'Grande superficie',
 'CARULLA COUNTRYB':'Grande superficie',
 'CARULLA FRESH MARKET RINCON COLINA':'Grande superficie',
 'CARULLA LA SOLEDAD':'Grande superficie',
 'CARULLA PABLO VI':'Grande superficie',
 'CARULLA PEPE SIERRA':'Grande superficie',
 'CARULLA RINCON':'Grande superficie',
 'CARULLA SOLEDAD':'Grande superficie',
 'CASA IDEAS':'Hogar',
 'CASA LIS':'Alimentación',
 'CASAIDEAS':'Hogar',
 'CASCABEL':'Alimentación',
 'CENCOSUD COLOMBIA':'Grande superficie',
 'CENTRO COMERCIAL LA CASONA':'Transporte',
 'CHEVIGNON':'Moda',
 'CHIPRE VIVA ENVIGADO':'Alimentación',
 'CHURRISIMO':'Alimentación',
 'CIA. MEDIC PREPAG COLSANITAS':'Salud y Bienestar',
 'CIA. MEDIC. PREPAG. COLSANITAS':'Salud y Bienestar',
 'CIATRAN':'Educación',
 'CINE COLOMBIA':'Entretenimiento',
 'CINEMARK COLOMBIA':'Entretenimiento',
 'CITY PARKING':'Transporte',
 'CLUB BOYACA':'Transporte',
 'CLUB GANADERO LA ESMERALDA':'Alimentación',
 'COLOMBINA':'Alimentación',
 'COLSUBSIDIO':'Grande superficie',
 'COLSUBSIDIO CRUZ ROJA':'Salud y Bienestar',
 'COMERCIAL PAPELERA':'Educación',
 'COMERCIALIZADORA BALDINI BALDINI':'Moda',
 'COMERCIALIZADORA NATURAL LIGHT':'Salud y Bienestar',
 'COMIDAS VARPEL / HOME BURGUERS':'Alimentación',
 'COMODIN':'Moda',
 'CONCESIONBANA DE OCCIDENTE':'Transporte',
 'CONSECIONBANA DE OCCIDENTE':'Transporte',
 'COOP OMEGA':'Transporte',
 'COOPERATIVA DESPORTADORES FLOTA':'Transporte',
 'COPSERVIR':'Salud y Bienestar',
 'CORRERIAS INVERSIONES':'Transporte',
 'CREPES AND WAFFLES':'Alimentación',
 'CREPES Y WAFFLES':'Alimentación',
 'CRUZ VERDE':'Salud y Bienestar',
 'D1.':'Grande superficie',
 'DECATHLON':'Salud y Bienestar',
 'DEPOSITO PRINCIPAL DE DROGAS':'Salud y Bienestar',
 'DERMATODO':'Salud y Bienestar',
 'DIALIMENTOS':'Alimentación',
 'DISSEN':'Moda',
 'DISTRACOM LA COROCORA':'Transporte',
 'DISTRIBUIDORA DONA ELENA':'Alimentación',
 'DISTRIBUIDORA DOÑA ELENA':'Alimentación',
 'DISTRIBUIDORA DOÑA ELENABOGOTA)':'Alimentación',
 'DISTRIBUIDORA PASTEUR':'Alimentación',
 'DISTRICARNES CAMACHO':'Alimentación',
 'DISTRITO 77':'Alimentación',
 'DLK':'Alimentación',
 'DOCUMENTO SUMINISTRO DE PRODUCTOS CRUZ VERDE':'Salud y Bienestar',
 'DOG KAT':'Mascotas',
 'DOGKAT':'Mascotas',
 'DOLLARCITY':'Grande superficie',
 'DONUCOL':'Alimentación',
 'DORANDO COLOMBIA':'Alimentación',
 'DOTBAND COLOMBIA':'Alimentación',
 'DROG LAS VILLAS DE COLOMBIA':'Salud y Bienestar',
 'DROGAS LA 80':'Salud y Bienestar',
 'DROGAS LA FRAGUA':'Salud y Bienestar',
 'DROGAS LA REBAJA':'Salud y Bienestar',
 'DROGAS MARANTA':'Salud y Bienestar',
 'DROGUERIA LA 80':'Salud y Bienestar',
 'DROGUERIA LA REBAJA':'Salud y Bienestar',
 'DROGUERIAS COLSUBSIDIO':'Salud y Bienestar',
 'DROGUERIAS CRUZ VERDE':'Salud y Bienestar',
 'DROGUERIAS Y FARMACIAS CRUZ VERDE':'Salud y Bienestar',
 'ECONOMAS':'Grande superficie',
 'EDICIONES URANO COLOMBIA':'Educación',
 'EL CHOLI SE MUDA A MEDELLIN':'Entretenimiento',
 'EL CORRAL GOURMET':'Alimentación',
 'EL DESPACHO VARIANTE MARIA FANNY RAMOS':'Alimentación',
 'EL GRANJERO ALIADO A SURTIMAX':'Alimentación',
 'EL HORNO DE PIEDRA':'Alimentación',
 'EL NECAR LA ORIGINAL':'Alimentación',
 'EL PAISATON':'Hogar',
 'EL PALACIO DE LA GALLINA EL POLITICO':'Alimentación',
 'EL PALACIO DE LAS OFERTAS':'Grande superficie',
 'EL REY DE LA LOCURA':'Grande superficie',
 'EL TAMBOR':'Alimentación',
 'ELA PLAZA DE LAS AMERICAS':'Moda',
 'ELIMUR':'Hogar',
 'EPSNITAS.':'Salud y Bienestar',
 'ESTRELLA ANDINA':'Alimentación',
 'EUROS SERVICES':'Grande superficie',
 'EXITO':'Grande superficie',
 'EXITO AMERICAS':'Grande superficie',
 'EXITO ENVIGADO':'Grande superficie',
 'EXITO ENVIGADO WOW':'Grande superficie',
 'EXITO EXPRESS CARRERA 15 CON 100':'Grande superficie',
 'EXITO FLORESTA':'Grande superficie',
 'EXITO GRAN ESTACION':'Grande superficie',
 'EXITO LA CENTRAL':'Grande superficie',
 'EXITO TUNJA':'Grande superficie',
 'EXITO VIVA TUNJA':'Grande superficie',
 'EXITO ZIPAQUIRA':'Grande superficie',
 'EXPERTOS EN CAFE':'Alimentación',
 'EXPLORANDES':'Alimentación',
 'FALABELLA':'Grande superficie',
 'FARMACIA FARMATODO':'Salud y Bienestar',
 'FARMATODO':'Salud y Bienestar',
 'FDS':'Moda',
 'FERRETERIA LA GENERAL':'Hogar',
 'FERRILLANOS':'Hogar',
 'FINCA URBANA PETS':'Mascotas',
 'FRANQUICIAS Y CONCESIONES':'Alimentación',
 'FRESH SODAS Y LIMONADAS':'Alimentación',
 'FRIGO CARNES LA 45':'Alimentación',
 'FRIGOCARNES LA 45':'Alimentación',
 'FRISBY':'Alimentación',
 'FRUTIVERDURAS CAMACHO':'Alimentación',
 'FRUTIVERDURASNTI':'Alimentación',
 'FRUTOS PLAZA':'Alimentación',
 'FRUTOS Y ESPECIAS EL CAUNZAL':'Alimentación',
 'FRUTOVERDURAS CAMACHO':'Alimentación',
 'FRUTTIVOS':'Alimentación',
 'FRUTY VERDURAS CAMACHO':'Alimentación',
 'FRUVAR':'Alimentación',
 'FRUVAR PONTEVEDRA':'Alimentación',
 'FXA':'Moda',
 'GANSO Y CASTOR MUSEO':'Entretenimiento',
 'GASTRONOMIA ITALIANA':'Alimentación',
 'GELSA':'Hogar',
 'GNE SOLUCIONES':'Alimentación',
 'GRAMA ENVIGADO':'Alimentación',
 'GRATIN BURGER':'Alimentación',
 'GRECOL':'Alimentación',
 'GRUPO AL ALMA':'Alimentación',
 'GRUPO EMPRESARIAL':'Alimentación',
 'GRUPO REPUBLIKA.':'Alimentación',
 'GRUPO SIERRA NEVADA':'Alimentación',
 'GRUPO VELAND':'Moda',
 'GUILLERMO PULGARIN S':'Moda',
 'H&M':'Moda',
 'H&S FRANQUICIAS':'Moda',
 'HACIENDA CIUDAD DEL RIO':'Alimentación',
 'HAMBUR LA NACIONAL':'Alimentación',
 'HAMBURGUESAS EL CORRAK':'Alimentación',
 'HAMBURGUESAS EL CORRAL':'Alimentación',
 'HAMBURGUESERIAS':'Alimentación',
 'HELADOS POPSI':'Alimentación',
 'HELADOS POPSY':'Alimentación',
 'HICO FISH':'Alimentación',
 'HIPERMAR FISH':'Alimentación',
 'HOLDING GASTRONOMICO':'Alimentación',
 'HOME BURGERS':'Alimentación',
 'HOME SENTRY':'Grande superficie',
 'HOMECENTER':'Grande superficie',
 'HOMECENTER CALLE 80':'Grande superficie',
 'HOMECENTER TINTAL':'Grande superficie',
 'HOMECENTER TUNJA':'Grande superficie',
 'HOTELES DE LA ESPERANZA':'Entretenimiento',
 'HOTELES LA ESPERANZA':'Entretenimiento',
 'HUB MOVILIDAD PLAZA':'Hogar',
 'HUMBERTO ELIECER PEREZ OCHOA':'Alimentación',
 'IBIZA DIGITAL':'Moda',
 'INCOEXPRES':'Alimentación',
 'INTECHNOLOGY':'Tecnología',
 'INTERCREDITO DE COLOMBIA':'Compromisos Bancarios',
 'INVERSIONES FARMADROGA':'Salud y Bienestar',
 'INVERSIONES FOGON Y LEÑA':'Alimentación',
 'INVERSIONES INT COLOMBIA':'Alimentación',
 'INVERSIONES JOMORA S. AS.':'Mascotas',
 'INVERSIONES MOCHI':'Alimentación',
 'INVERSIONES MONTOLIVO':'Alimentación',
 'INVERSIONES NECEVI':'Alimentación',
 'INVERSIONES SOM':'Alimentación',
 'INVERSIONES SUPER EXPRESS':'Alimentación',
 'INVERSIONES SUPERVAQUITA':'Compromisos Bancarios',
 'IRCC':'Alimentación',
 'ISHOP COLOMBIA':'Tecnología',
 'JARAMILLO & JARAMILLO':'Alimentación',
 'JOSE MARIA':'Grande superficie',
 'JUICY LUCY':'Alimentación',
 'JUMBO':'Grande superficie',
 'JUNGLA PARK BOLERA':'Alimentación',
 'KAPITAL PLUS COLOMBIA':'Alimentación',
 'KARLUZ':'Hogar',
 'KIRFA GOLD INTERNATIONAL':'Moda',
 'KOAJ':'Moda',
 'KOBA COLOMBIA':'Grande superficie',
 'LA BIFERIA':'Alimentación',
 'LA CANASTA CAMPESINA':'Alimentación',
 'LA COLMENA DE HOMECENTER':'Hogar',
 'LA CORALINA':'Transporte',
 'LA GRAN COSECHA':'Alimentación',
 'LA MIGUERIA':'Alimentación',
 'LA MONFERRINA':'Alimentación',
 'LA PAPELERIA Y CIA':'Educación',
 'LA PARRILLA "LA ORIGINAL" AMERICAS':'Alimentación',
 'LA PINTA':'Alimentación',
 'LA PROPIA TAQUERIA':'Alimentación',
 'LACOSTE':'Moda',
 'LAFAM':'Salud y Bienestar',
 'LAVASECO FINAS TAMPA':'Hogar',
 'LEGIS':'Educación',
 'LEYER EDITORES':'Educación',
 'LIBRERIA NACIONAL':'Educación',
 'LINEAS DEL VALLE':'Transporte',
 'LOCATEL':'Salud y Bienestar',
 'LONCHERIA DC':'Alimentación',
 'LOS HORNITOS PANADERIA Y PASTELERIA':'Alimentación',
 'LOS HORNITOS PASTELERIA Y PANADERIA':'Alimentación',
 'MACROSURTIDO-MUNDO HUEVO':'Grande superficie',
 'MADAME LILI':'Alimentación',
 'MADECENTRO':'Hogar',
 'MAGASIN':'Hogar',
 "MAGICNAIL'S":'Salud y Bienestar',
 'MAKRO':'Grande superficie',
 'MANKA DEL PERU':'Alimentación',
 'MARLON BECERRA':'Salud y Bienestar',
 'MASFINCA.':'Mascotas',
 'MCCOY PLAZA LAS AME':'Alimentación',
 'MECANELECTRO':'Hogar',
 'MEETING HOUSE':'Alimentación',
 'MERCADERIA JUSTO & BUENO':'Grande superficie',
 'MERCADERIA JUSTO Y BUENO':'Grande superficie',
 'MERCADO ZAPATOCA':'Grande superficie',
 'MERCADOS BLANCO TORRES':'Alimentación',
 'MERCADOS LIDERN CIPRIANO':'Alimentación',
 'MERCASOCIAL VERBENAL':'Alimentación',
 'MESA FRANCA':'Alimentación',
 'MILSABORES':'Alimentación',
 'MINIMARKET EL GRAN LIMONAR HUMBERTO RODRIGUEZ SILVA':'Alimentación',
 'MINISO COLOMBIA':'Hogar',
 'MUNDO VERDE':'Alimentación',
 'NAPPA':'Moda',
 'NIKOLUKAS':'Alimentación',
 'NINE FIFTY RESTO BAR':'Alimentación',
 'NOTARIA 28':'Impuestos',
 'NOTARIA QUINCE':'Impuestos',
 'OBOTICARIO':'Salud y Bienestar',
 'OFICINA PRINCIPAL ASSASING':'Alimentación',
 'OFICINA PRINCIPAL ESMERALDA EXPRESS':'Alimentación',
 'OLIMMPICA':'Grande superficie',
 'OLIMPICA':'Grande superficie',
 'OLIVENZA':'Alimentación',
 'OMA':'Alimentación',
 'OPERADORA FRANQUICIAS DE COLOMBIA':'Alimentación',
 'OPSERVIR':'Salud y Bienestar',
 'ORQUIDEA':'Alimentación',
 'OUTLET MARCA SOCIA':'Moda',
 'OXXO':'Grande superficie',
 'PAN PA YA':'Alimentación',
 'PANADERIA Y PASTELERIA LA FUENTE DORADA':'Educación',
 'PANAMERICANA':'Educación',
 'PANAMERICANA LIBRERIA Y PAPELERIA':'Educación',
 'PARKING':'Transporte',
 'PARKING 18':'Transporte',
 'PARKING INTERNACIONAL':'Transporte',
 'PARKING INTERNATIONAL':'Transporte',
 'PARQUEADERO DEL COUNTRY':'Transporte',
 'PCA PRODUCTORA Y COMERCIALIZADORA DE ALIMENTOS':'Alimentación',
 'PEAJE BICENTENARIO':'Transporte',
 'PEAJE HONDA':'Transporte',
 'PENGUIN RANDOM HOUSE':'Educación',
 'PERCIMON':'Alimentación',
 'PERMODA':'Moda',
 'PIGASUS ENVIGADO':'Alimentación',
 'PIZZA HUT':'Alimentación',
 'POUND':'Alimentación',
 'PREGO':'Alimentación',
 'PREMIUM PORTALES':'Alimentación',
 'PRESTO':'Alimentación',
 'PRICESMART':'Grande superficie',
 'PRICESMART ELLVADOR':'Grande superficie',
 'DE CV':'Pendiente',
 'PRIMELL GROUP':'Moda',
 'PROCAFE COL':'Alimentación',
 'PROCAFECOL':'Alimentación',
 'PROCESADORA DE ALIMENTOS EL GORDO':'Alimentación',
 'PROFAMILIA':'Salud y Bienestar',
 'PROKARTING':'Entretenimiento',
 'PROYECTOS RECREATIVOS':'Entretenimiento',
 'PTEQUENDAMA':'Transporte',
 'PUMA COLINA':'Moda',
 'PUNTO BLANCO':'Moda',
 'PUPPIS':'Mascotas',
 'QBANO VILLA DEL PRADO':'Alimentación',
 'RAFAEL J. TURBAY':'Moda',
 'RANCHO LLANERO':'Alimentación',
 'RCADOS BLANCO TORRES':'Alimentación',
 'RED EMPRESARIA DE SERVICIOS':'Compromisos Bancarios',
 'RESOLUCION DE FACTURACION POR COMPUTADOR':'Alimentación',
 'RESTAURANTE LA MOLIENDA DE TAVO':'Alimentación',
 'RESTCAFE':'Alimentación',
 'RIB EYE PARRILLA & BAR':'Alimentación',
 'ROLL COMPANY':'Alimentación',
 'SABOR LLANERO':'Alimentación',
 'SALSAMENTARIA CAMACHO':'Alimentación',
 'SALSAMENTARIA LA JAPONESA':'Alimentación',
 'SANITAS':'Salud y Bienestar',
 'SANTA LEÑA':'Alimentación',
 'SILVANO ALDO SICILIA GUZZI':'Moda',
 'SISTEMA P. O. S. COLSUBSIDIO':'Salud y Bienestar',
 'SOCIEDAD DESPOSTADORA DE CARNES':'Alimentación',
 'SOMOS MASA':'Alimentación',
 'SOY MAYZ':'Alimentación',
 'SPEEDO':'Moda',
 'STARBUCKS':'Alimentación',
 'STARBUCKS CITY U':'Alimentación',
 'STD':'Moda',
 'STF GROUP':'Moda',
 'STUDIO F PLAZA DE LAS AMERICAS':'Moda',
 'SUBWAY':'Alimentación',
 'SUPERMERCADO EL PORVENIR':'Alimentación',
 'SUPERMERCADO LA PORTEÑA SYG':'Alimentación',
 'SUPERMERCADOS COMUNAL':'Alimentación',
 'SUSHI MARKET':'Alimentación',
 'TANIA':'Moda',
 'TAX META':'Transporte',
 'TENTACIONES DEL NECTAR':'Alimentación',
 'TERMINAL DESPORTE':'Transporte',
 'THE PARKING':'Transporte',
 'THERMOLINE FERRETERIA':'Alimentación',
 'TIENDAS ARA':'Grande superficie',
 'TODO DROGAS 9':'Salud y Bienestar',
 'TODO REMATE':'Grande superficie',
 'TOLEDO PASTELERIA':'Alimentación',
 'TOSTAO':'Alimentación',
 'TOTTO':'Moda',
 'TPVM10101':'Transporte',
 'TRENDY SHOP BTA':'Salud y Bienestar',
 'TURBO CARULLA':'Grande superficie',
 'UNIDROGAS':'Salud y Bienestar',
 'VEREDA LA SELVA KM 4 CRISTO REY':'Alimentación',
 'VIA BALOTO':'Compromisos Bancarios',
 'VILLA DEL PRADO':'Grande superficie',
 'VIVERO TIERRA NEGRA':'Hogar',
 'VORAZ':'Alimentación',
 'WAJACA':'Alimentación',
 'WINGION':'Alimentación',
 'WING STATION':'Alimentación',
 'WOK CERDITOS':'Alimentación',
 'WOK COLINA':'Alimentación',
 'WOK IZAKAYA':'Alimentación',
 'WOK MUSEO':'Alimentación',
 'WWW.PTEQUENDAMA.COM':'Transporte',
 'YOI':'Hogar',
 'ZARA UNICENTRO':'Moda',
 'ZX VENTURES COLOMBIA':'Alimentación',
 'DISTRIBUIDORA DOÑA ELENABOGOTA)':'Alimentación',
 'FRUTIVERDURAS CAMACHODURAS CAMACH9':'Alimentación',
 'FRUTIVERDURAS CAMACHODURAS CAMACHO':'Alimentación',
 'FRUTIVERDURAS CAMACHODURASNTI':'Alimentación',
 'PRICESMART ELLVADOR, DE CV':'Grande superficie',
 'AGROCAMPO': 'Mascotas',
'ALMACEN RIAÑO - TUNJA': 'Educación',
'BEAUTY SHOP': 'Salud y Bienestar',
'BELLA PIEL': 'Salud y Bienestar',
'BOTICARIO COLOMBIA': 'Salud y Bienestar',
'CENCOSUD': 'Grande superficie',
'CENTRO COMERCIAL PLAZA 54': 'Transporte',
'CERAGRO': 'Mascotas',
'CLUB DEL VINO': 'Alimentación',
'CREPES & WAFFLES': 'Alimentación',
'DON BENITEZ': 'Alimentación',
'DROGUERIA MINIMARKET': 'Salud y Bienestar',
'DROGUERIA MUNDIAL': 'Salud y Bienestar',
'EDITORIAL PLANETA': 'Educación',
'FACTURA POS GRANJA MARINA FISH MARKET': 'Alimentación',
'FARMALKOSTO DROGUERIA PINAR': 'Salud y Bienestar',
'FARMATODO COLOMBIA': 'Salud y Bienestar',
'FIORELLA PIZZA': 'Alimentación',
'FRUTIVERDURAS CAMACHO 1A': 'Alimentación',
'JUAN VALDEZ': 'Alimentación',
'KFC': 'Alimentación',
'KTRONIX': 'Tecnología',
'LA LUCHANGUCHERIA CRIOLLA': 'Alimentación',
'MABEL ROCIO ARIAS GARAY': 'Salud y Bienestar',
'MINISO': 'Hogar',
'MUY': 'Alimentación',
'PASTAIO': 'Alimentación',
'RESCAFE': 'Alimentación',
'RICA CARNE CENTRO':'Alimentación',
'SISTEMA POS':'Alimentación',
'STRADIVARIUS':'Moda',
'SURTIFRUVER LA CANASTA DE MI BARRIO':'Alimentación',
'TORNAMESA ZONA G': 'Educación',
'UNICENTRO TUNJA':'Transporte',
'VILLA ROMANA':'Moda'
}


In [6]:
def categoría():
    
    df = prepro()
    
    df_facturas = df[df['doc_type'] == 'invoice']
    df_no_facturas = df[df['doc_type'] == 'no-invoice']
    
    df_facturas['Categoria'] = df_facturas['issuer'].map(diccionario) 
    df_no_facturas['Categoria'] = "No factura"
    

    return pd.concat([df_facturas, df_no_facturas], axis=0)
    

In [7]:
# Leer el dataset actualizado:
def getLatestDataset():
    
    return categoría()

In [8]:
# Esta función nos ayuda a elegir un registro aleatorio en el dataset para hacer validaciones:
def getRandomDocumentFromDB():
    df = pd.read_csv(LATEST_CSV)
    doc_list = df.filename
    return random.choice(doc_list)

In [9]:
# Esta función nos ayuda a elegir un registro aleatorio en el dataset para hacer validaciones, siempre y cuando sea factura:
def getRandomInvoiceFromDB():
    df = pd.read_csv(LATEST_CSV)
    doc_list = df[df.doc_type == 'invoice'].filename
    return random.choice(doc_list)

In [10]:
# Esta función nos ayuda a elegir un registro aleatorio en el dataset para hacer validaciones, siempre y cuando NO sea factura:
def getRandomNoInvoiceFromDB():
    df = pd.read_csv(LATEST_CSV)
    doc_list = df[df.doc_type == 'no-invoice'].filename
    return random.choice(doc_list)

In [11]:
# Esta función nos ayuda a extraer el total del texto en una imágen, en caso de existir texto:
def getInvoiceTotalWithTextract(filename):
    client = boto3.client('textract', region_name=REGION_NAME)
    response = client.analyze_expense(Document={'S3Object': {'Bucket': BUCKET, 'Name': filename}})
    
    for summary_field in response['ExpenseDocuments'][0]['SummaryFields']:
        if "Type" in summary_field and summary_field['Type']['Text'] == 'TOTAL':
            return(summary_field['ValueDetection']['Text'])
    return('0')

In [12]:
# Esta función nos ayuda a eliminar palabras y caracteres que no queremos en el dataset:

def cleandf(df):

    #PRIMERO VAMOS A PONERLAS EN MAYÚSCULAS
    df['issuer'] = df['issuer'].str.strip(' ')
    df['issuer'] = df['issuer'].str.upper()

    #REEMPLAZAMOS LAS TILDES POR EL CARACTER
    df['issuer'] = df['issuer'].replace({'Á':'A'}, regex=True)
    df['issuer'] = df['issuer'].replace({'É':'E'}, regex=True)
    df['issuer'] = df['issuer'].replace({'Í':'I'}, regex=True)
    df['issuer'] = df['issuer'].replace({'Ó':'O'}, regex=True)
    df['issuer'] = df['issuer'].replace({'Ú':'U'}, regex=True)

    #REEMPLAZAMOS EL DOBLE ESPACIO
    df['issuer'] = df['issuer'].replace({'  ':' '}, regex=True)

    #ELIMINAMOS ESAS PALABRAS
    df['issuer'] = df['issuer'].replace({' LTDA.':''}, regex=True)
    df['issuer'] = df['issuer'].replace({' LTDA':''}, regex=True)
    df['issuer'] = df['issuer'].replace({' SAS':''}, regex=True)
    df['issuer'] = df['issuer'].replace({' S.A.S':''}, regex=True)
    df['issuer'] = df['issuer'].replace({' S.A.S.':''}, regex=True)
    df['issuer'] = df['issuer'].replace({' S. A. S.':''}, regex=True)
    df['issuer'] = df['issuer'].replace({' S. A. S':''}, regex=True)
    df['issuer'] = df['issuer'].replace({' T.A.':''}, regex=True)
    df['issuer'] = df['issuer'].replace({' SA':''}, regex=True)
    df['issuer'] = df['issuer'].replace({' S.A.':''}, regex=True)
    df['issuer'] = df['issuer'].replace({' S.A':''}, regex=True)

    #ARREGLAMOS EL DATASET CON PALABRAS PARA DEJAR UNA BUENA CALIDAD DE IFO
    df['issuer'] = df['issuer'].replace({'BBI COLOMBIA.':'BBI COLOMBIA'}, regex=True)
    df['issuer'] = df['issuer'].replace({'BBI COLOMBIA.':'BBI COLOMBIA'}, regex=True)
    df['issuer'] = df['issuer'].replace({'CADENA COMERCIAL OXXO COLOMBIA':'CADENA COMERCIAL OXXO'}, regex=True)
    df['issuer'] = df['issuer'].replace({'CADENA COMERCIAL OXXO':'CADENA COMERCIAL OXXO COLOMBIA'}, regex=True)
    df['issuer'] = df['issuer'].replace({'CARILLA ':'CARULLA '}, regex=True)
    df['issuer'] = df['issuer'].replace({'COLDANITAS':'COLSANITAS'}, regex=True)
    df['issuer'] = df['issuer'].replace({'COLDANITAS':'COLSANITAS'}, regex=True)
    df['issuer'] = df['issuer'].replace({'D1.':'D1'}, regex=True)
    df['issuer'] = df['issuer'].replace({'D1':'D1.'}, regex=True)
    df['issuer'] = df['issuer'].replace({'EPSNITAS.':'EPSNITAS'}, regex=True)
    df['issuer'] = df['issuer'].replace({'EPSNITAS':'EPSNITAS.'}, regex=True)


    # LISTA DE COMERCIOS:
    #prueba = df.sort_values(by="issuer")
    #prueba = prueba['issuer'].unique()

    #print(prueba)


    return(df)
    

In [13]:
def printDoc(bucket, document, region):
    s3_connection = boto3.resource('s3')
    s3_object = s3_connection.Object(bucket, document)
    s3_response = s3_object.get()
    stream = io.BytesIO(s3_response['Body'].read())
    image = Image.open(stream)
    width, height = image.size
    draw = ImageDraw.Draw(image)
    image.show()